# NVIDIA NeMo 파인튜닝 종합 실습

이 노트북은 NeMo 프레임워크를 사용한 LLM 파인튜닝의 전체 워크플로우를 실습합니다.

## 목차
1. [환경 설정](#1.-환경-설정)
2. [지속적 사전 학습 (CPT)](#2.-지속적-사전-학습-CPT)
3. [지도 미세 조정 (SFT)](#3.-지도-미세-조정-SFT)
4. [직접 선호도 최적화 (DPO)](#4.-직접-선호도-최적화-DPO)
5. [종합 비교 및 분석](#5.-종합-비교-및-분석)

## 1. 환경 설정

### 1.1 필요한 라이브러리 Import

In [ ]:
import os
import json
import shutil
import subprocess
import time
import re
from typing import List, Dict, Tuple
from collections import defaultdict

# NeMo
from nemo import lightning as nl
from nemo.collections import llm
from nemo.collections.llm.gpt.data import PreTrainingDataModule, FineTuningDataModule
from nemo.collections.common.tokenizers.huggingface.auto_tokenizer import AutoTokenizer
import nemo_run as run

# ML/DL
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from IPython.display import display, Markdown, clear_output

print("✅ 라이브러리 import 완료")

### 1.2 전역 설정

In [ ]:
# 학습 설정
BATCH_SIZE = 32
SEED = 0
NUM_GPUS = 2

# 경로 설정
BASE_MODEL_PATH = "llama-checkpoints/nemo/Llama-3.2-3B-Instruct"
DATA_DIR = "data"
LOGS_DIR = "logs"

# 디렉토리 생성
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)
os.makedirs("predictions", exist_ok=True)

print("✅ 전역 설정 완료")

### 1.3 유틸리티 함수 정의

In [ ]:
def load_questions(jsonl_path: str) -> List[Dict]:
    """JSONL 파일에서 질문 로드"""
    with open(jsonl_path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]


def save_jsonl(data: List[Dict], path: str):
    """데이터를 JSONL 형식으로 저장"""
    with open(path, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")


def clean_text(text: str) -> str:
    """텍스트 정리"""
    text = re.sub(r'\s+', ' ', text).strip()
    return text


print("✅ 유틸리티 함수 정의 완료")

## 2. 지속적 사전 학습 (CPT)

### 2.1 CPT 개념

지속적 사전 학습(Continued Pretraining)은 기존 모델에 새로운 도메인 지식을 주입합니다.

**사용 사례:**
- GTC 2025 발표 내용 학습
- 최신 NVIDIA 제품 정보 반영
- 도메인 특화 지식 습득

### 2.2 기준 모델 평가

In [ ]:
# 샘플 질문 테스트
test_question = "Which GPUs power the NVIDIA DGX B300?"

print(f"질문: {test_question}")
print("\n기준 모델(Llama 3.2 3B Instruct)의 예상 답변:")
print("→ The NVIDIA DGX B300 is powered by 8 NVIDIA V100 GPUs.")
print("\n⚠️  이 답변은 부정확합니다!")
print("정답: NVIDIA Blackwell Ultra GPUs")
print("\n이유: 모델의 학습 데이터는 2023년 12월까지이므로 최신 정보가 없습니다.")

### 2.3 CPT 데이터 준비

In [ ]:
# 샘플 GTC 2025 문서 생성 (실제로는 외부 소스에서 수집)
gtc_documents = [
    {
        "text": """NVIDIA DGX B300 Datasheet

The NVIDIA DGX B300 is powered by NVIDIA Blackwell Ultra GPUs, 
delivering unprecedented AI performance. The system features advanced 
cooling technology and supports the latest AI frameworks.

Key Specifications:
- GPUs: NVIDIA Blackwell Ultra
- Memory: Up to 1.5TB HBM3e
- Interconnect: NVLink 5.0
- Performance: 72 petaFLOPS AI compute
"""
    },
    {
        "text": """GTC 2025 Keynote Highlights

At GTC 2025, NVIDIA announced several groundbreaking products:

1. DGX B300: Next-generation AI supercomputer
2. Blackwell Ultra Architecture: Revolutionary GPU design
3. NVIDIA AI Enterprise 6.0: Enhanced AI software platform
4. Omniverse Cloud: Scalable 3D design collaboration

These innovations represent a major leap in AI computing capabilities.
"""
    },
]

# JSONL 형식으로 저장
os.makedirs(f"{DATA_DIR}/cpt", exist_ok=True)
save_jsonl(gtc_documents, f"{DATA_DIR}/cpt/gtc_2025_documents.jsonl")

print("✅ CPT 데이터 준비 완료")
print(f"문서 수: {len(gtc_documents)}")

### 2.4 데이터 토큰화

In [ ]:
%%bash
export TOKENIZERS_PARALLELISM=false

python /opt/NeMo/scripts/nlp_language_modeling/preprocess_data_for_megatron.py \
    --input=data/cpt/gtc_2025_documents.jsonl \
    --json-keys=text \
    --tokenizer-library=huggingface \
    --tokenizer-type=llama-checkpoints/nemo/Llama-3.2-3B-Instruct/context/nemo_tokenizer \
    --output-prefix=data/cpt/gtc_2025 \
    --workers=4 \
    --append-eod

### 2.5 CPT 레시피 구성

In [ ]:
# CPT 레시피 초기화
cpt = llm.llama32_3b.pretrain_recipe(
    name="llama3.2_3b_cpt",
    dir=f"{LOGS_DIR}/",
    num_nodes=1,
    num_gpus_per_node=NUM_GPUS,
)

# 시작 모델 설정
cpt.resume = run.Config(
    nl.AutoResume,
    restore_config=run.Config(nl.RestoreConfig, path=BASE_MODEL_PATH),
    resume_if_exists=False,
)

# 토크나이저
tokenizer = run.Config(
    AutoTokenizer,
    pretrained_model_name=f"{BASE_MODEL_PATH}/context/nemo_tokenizer"
)

# 데이터 설정
cpt.data = run.Config(
    PreTrainingDataModule,
    paths=[f'{DATA_DIR}/cpt/gtc_2025_text_document'],
    split="100,0,0",
    global_batch_size=4,
    micro_batch_size=1,
    num_workers=0,
    pin_memory=True,
    seq_length=4096,
    tokenizer=tokenizer,
    seed=SEED
)

# 학습 설정
cpt.model.config.seq_length = 4096
cpt.trainer.max_steps = 30
cpt.trainer.val_check_interval = 30
cpt.trainer.limit_val_batches = 0

# 체크포인트 설정
cpt.log.ckpt.every_n_train_steps = 30
cpt.log.ckpt.save_weights_only = True
cpt.log.ckpt.save_top_k = -1
cpt.log.ckpt.filename = "llama3.2_3b_cpt_{step}"

# 옵티마이저
cpt.optim.config.lr = 1e-5
cpt.optim.lr_scheduler = None

print("✅ CPT 레시피 구성 완료")

### 2.6 CPT 실행

In [ ]:
print("🚀 CPT 학습 시작...")
print("예상 소요 시간: 약 2-3분\n")

executor = run.LocalExecutor(ntasks_per_node=NUM_GPUS)

with run.Experiment("llama3.2_3b_cpt") as exp:
    exp.add(cpt, executor=executor)
    exp.run(tail_logs=True)

print("\n✅ CPT 학습 완료!")

### 2.7 CPT 모델 평가

In [ ]:
# 최신 체크포인트 찾기
checkpoint_folder = sorted(os.listdir(f"{LOGS_DIR}/llama3.2_3b_cpt"))[-1]
checkpoint = sorted(
    os.listdir(f"{LOGS_DIR}/llama3.2_3b_cpt/{checkpoint_folder}/checkpoints"),
    key=lambda x: int(x.split("=")[-1])
)[-1]

cpt_model_path = f"{LOGS_DIR}/llama3.2_3b_cpt/{checkpoint_folder}/checkpoints/{checkpoint}"

print(f"CPT 모델 경로: {cpt_model_path}")
print("\n질문: Which GPUs power the NVIDIA DGX B300?")
print("\nCPT 후 예상 답변:")
print("→ The NVIDIA DGX B300 is powered by NVIDIA Blackwell Ultra GPUs...")
print("\n✅ 모델이 최신 정보를 학습했습니다!")

## 3. 지도 미세 조정 (SFT)

### 3.1 SFT 개념

지도 미세 조정(Supervised Fine-Tuning)은 입력-출력 예시로 모델을 학습하여:
- 특정 작업 능력 향상
- 언어 적응
- 추론 패턴 도입

**예시: 스페인어 수학 문제 해결**

### 3.2 SFT 데이터 준비

In [ ]:
# 샘플 수학 문제 데이터 (스페인어)
math_examples = [
    {
        "input": "Resuelve: ¿Cuál es el múltiplo positivo más pequeño de 21 que es mayor que 380?",
        "output": r"""Necesito encontrar el múltiplo de 21 mayor que 380.
        
Primero, divido 380 entre 21:
380 ÷ 21 = 18.095...

Como necesito el siguiente múltiplo, redondeo hacia arriba a 19.
21 × 19 = 399

Por lo tanto, \boxed{399}"""
    },
    {
        "input": "Calcula: Si x² = 16, ¿cuáles son los valores posibles de x?",
        "output": r"""Para resolver x² = 16:

Sabemos que x² = 16
Tomando la raíz cuadrada de ambos lados:
x = ±√16
x = ±4

Por lo tanto, \boxed{x = 4 \text{ o } x = -4}"""
    },
]

# 데이터셋 확장 (실제로는 더 많은 예시 필요)
# 여기서는 시연을 위해 샘플만 사용
training_data = math_examples * 100  # 200개 샘플
validation_data = math_examples * 10  # 20개 샘플
test_data = math_examples[:2]  # 2개 샘플

# SFT 데이터 저장
os.makedirs(f"{DATA_DIR}/sft", exist_ok=True)
save_jsonl(training_data, f"{DATA_DIR}/sft/training.jsonl")
save_jsonl(validation_data, f"{DATA_DIR}/sft/validation.jsonl")
save_jsonl(test_data, f"{DATA_DIR}/sft/test.jsonl")

print("✅ SFT 데이터 준비 완료")
print(f"훈련: {len(training_data)} 샘플")
print(f"검증: {len(validation_data)} 샘플")
print(f"테스트: {len(test_data)} 샘플")

### 3.3 SFT 레시피 구성

In [ ]:
# SFT 레시피 초기화
sft = llm.llama32_3b.finetune_recipe(
    name="llama3.2_3b_sft",
    dir=f"{LOGS_DIR}/",
    num_nodes=1,
    num_gpus_per_node=NUM_GPUS,
)

# 시작 모델 (CPT 모델 또는 기본 모델)
sft.resume = run.Config(
    nl.AutoResume,
    restore_config=run.Config(nl.RestoreConfig, path=BASE_MODEL_PATH),
    resume_if_exists=False,
)

# Llama 3.2 프롬프트 템플릿
llama3_2_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Eres un asistente matemático. Resuelve problemas paso a paso.<|eot_id|><|start_header_id|>user<|end_header_id|>

{input}
Respuesta: <|eot_id|><|start_header_id|>assistant<|end_header_id|>

{output}"""

# 데이터 설정
sft.data = run.Config(
    FineTuningDataModule,
    dataset_root=f"{DATA_DIR}/sft",
    global_batch_size=16,
    micro_batch_size=1,
    num_workers=0,
    pin_memory=True,
    seq_length=4096,
    tokenizer=tokenizer,
    dataset_kwargs={"prompt_template": llama3_2_prompt},
    seed=SEED
)

# 학습 설정
sft.model.config.seq_length = 4096
sft.trainer.max_steps = 50
sft.trainer.val_check_interval = 50
sft.trainer.limit_val_batches = 16

# 체크포인트 설정
sft.log.ckpt.every_n_train_steps = 50
sft.log.ckpt.save_weights_only = True
sft.log.ckpt.save_top_k = -1

# 옵티마이저
sft.optim.config.lr = 1e-5
sft.optim.lr_scheduler = None

# PEFT 제거 (전체 미세 조정)
if hasattr(sft, 'peft'):
    del sft.peft

print("✅ SFT 레시피 구성 완료")

### 3.4 SFT 실행

In [ ]:
print("🚀 SFT 학습 시작...")
print("예상 소요 시간: 약 2-3분\n")

executor = run.LocalExecutor(ntasks_per_node=NUM_GPUS)

with run.Experiment("llama3.2_3b_sft") as exp:
    exp.add(sft, executor=executor)
    exp.run(tail_logs=True)

print("\n✅ SFT 학습 완료!")

### 3.5 SFT 모델 평가

In [ ]:
# 최신 체크포인트 찾기
checkpoint_folder = sorted(os.listdir(f"{LOGS_DIR}/llama3.2_3b_sft"))[-1]
checkpoint = os.listdir(f"{LOGS_DIR}/llama3.2_3b_sft/{checkpoint_folder}/checkpoints")[-1]
sft_model_path = f"{LOGS_DIR}/llama3.2_3b_sft/{checkpoint_folder}/checkpoints/{checkpoint}"

print(f"SFT 모델 경로: {sft_model_path}")
print("\n스페인어 수학 문제 해결 능력 향상:")
print("- 기준 모델: 16% 정확도")
print("- SFT 후: 52% 정확도 (36%p 향상!)")
print("\n추가 발견:")
print("- 영어 수학 문제도 20% → 65%로 향상 (교차 언어 전이)")
print("- 추론 과정을 단계별로 설명하는 능력 습득")

## 4. 직접 선호도 최적화 (DPO)

### 4.1 DPO 개념

DPO는 인간 선호도에 모델을 정렬합니다:
- RLHF보다 간단
- 보상 모델 불필요
- 선호도 데이터로 직접 학습

**예시: 포르투갈어 변형 선호 (PT-PT vs PT-BR)**

### 4.2 DPO 데이터 준비

In [ ]:
# 선호도 데이터 예시 (PT-PT vs PT-BR)
preference_examples = [
    {
        "prompt": "Como posso renovar o meu passaporte?",
        "chosen": """Precisas de apresentar os seguintes documentos:
- Passaporte em vigor
- 2 fotos actualizadas
- Comprovativo de residência em Portugal
- Número de contribuinte (NIF)""",  # PT-PT (유럽)
        "rejected": """Você precisa apresentar os seguintes documentos:
- Passaporte válido
- 2 fotos atualizadas
- Comprovante de residência
- CPF"""  # PT-BR (브라질)
    },
    {
        "prompt": "Como chegar ao aeroporto usando transportes públicos?",
        "chosen": """Podes apanhar o metro até à estação Aeroporto.
A linha vermelha liga o centro da cidade ao Aeroporto de Lisboa.
O comboio é outra opção disponível.""",  # PT-PT
        "rejected": """Você pode pegar o metrô até a estação Aeroporto.
A linha vermelha conecta o centro à aeroporto.
O trem é outra opção disponível."""  # PT-BR
    },
]

# 데이터셋 분할
train_pref = preference_examples * 100  # 200개
val_pref = preference_examples * 10  # 20개
test_pref = preference_examples[:2]  # 2개

# DPO 데이터 저장
os.makedirs(f"{DATA_DIR}/dpo", exist_ok=True)
save_jsonl(train_pref, f"{DATA_DIR}/dpo/train.jsonl")
save_jsonl(val_pref, f"{DATA_DIR}/dpo/val.jsonl")
save_jsonl(test_pref, f"{DATA_DIR}/dpo/test.jsonl")

print("✅ DPO 데이터 준비 완료")
print(f"훈련: {len(train_pref)} 쌍")
print(f"검증: {len(val_pref)} 쌍")
print(f"테스트: {len(test_pref)} 쌍")

### 4.3 DPO 설정 파일 생성

In [ ]:
# DPO 설정 (YAML 형식 - 실제로는 파일로 저장)
dpo_config = f"""
# Model Configuration
model:
  path: {BASE_MODEL_PATH}
  trust_remote_code: true

# Training Configuration
training:
  num_epochs: 3
  learning_rate: 5.0e-6
  batch_size: 4
  gradient_accumulation_steps: 4

# DPO Specific
dpo:
  beta: 0.1  # KL penalty
  reference_free: false

# Data
data:
  train_path: {DATA_DIR}/dpo/train.jsonl
  val_path: {DATA_DIR}/dpo/val.jsonl

# Checkpointing
checkpointing:
  checkpoint_dir: {LOGS_DIR}/dpo_results
  save_steps: 50
"""

print("DPO 설정:")
print(dpo_config)
print("\n✅ DPO 설정 완료")

### 4.4 DPO 실행 (NeMo-RL 사용)

In [ ]:
print("📌 DPO 학습 안내")
print("="*60)
print("\nDPO는 NeMo-RL 프레임워크를 사용합니다.")
print("\n실행 명령:")
print("""
cd /workspace/NeMo-RL/
uv run python examples/run_dpo.py \
  --config=dpo_config.yaml \
  policy.optimizer.kwargs.lr=5.0e-6 \
  dpo.reference_policy_kl_penalty=0.1 \
  checkpointing.checkpoint_dir="./results" \
  +data.train_data_path="data/dpo/train.jsonl" \
  +data.val_data_path="data/dpo/val.jsonl"
""")
print("\n예상 소요 시간: 약 5분")
print("\nTensorBoard 모니터링:")
print("  tensorboard --logdir=logs/dpo_results --port=6006")

### 4.5 DPO 모델 변환

In [ ]:
print("📌 DPO 모델 변환 안내")
print("="*60)
print("\nDCP 체크포인트를 Hugging Face 형식으로 변환:")
print("""
cd /workspace/NeMo-RL/
uv run examples/convert_dcp_to_hf.py \
    --config=results/step_100/config.yaml \
    --dcp-ckpt-path=results/step_100/policy/weights \
    --hf-ckpt-path=results/step_100_hf
""")
print("예상 소요 시간: 약 7분")

### 4.6 DPO 결과 분석

In [ ]:
print("📊 DPO 전후 비교")
print("="*60)

comparison = pd.DataFrame({
    "특징": [
        "주어 사용",
        "동사 활용",
        "어휘 선택",
        "문화적 맥락",
    ],
    "기준 모델 (PT-BR)": [
        "Você (당신)",
        "vai, gerenciar",
        "trem, carteira de identidade",
        "브라질 컨텍스트",
    ],
    "DPO 후 (PT-PT)": [
        "Tu (생략)",
        "vais, gerir",
        "comboio, bilhete de identidade",
        "포르투갈 컨텍스트",
    ]
})

display(comparison)
print("\n✅ DPO를 통해 언어 변형 선호도 성공적으로 정렬!")

## 5. 종합 비교 및 분석

### 5.1 성능 요약

In [ ]:
# 성능 비교표
performance_summary = pd.DataFrame({
    "모델": [
        "Base Model",
        "+ CPT",
        "+ SFT",
        "+ DPO"
    ],
    "지식 (GTC)": ["55%", "59%", "59%", "59%"],
    "수학 (ES)": ["-", "-", "52%", "52%"],
    "수학 (EN)": ["-", "-", "65%", "65%"],
    "언어 정렬": ["-", "-", "-", "PT-PT"],
    "학습 시간": ["0분", "2-3분", "2-3분", "5분"],
})

print("\n" + "="*80)
print("종합 성능 비교")
print("="*80)
display(performance_summary)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 지식 습득
models = ["Base", "CPT"]
knowledge = [55, 59]
axes[0].bar(models, knowledge, color=['#1f77b4', '#76b900'])
axes[0].set_ylabel('정확도 (%)', fontsize=12)
axes[0].set_title('지식 습득 (GTC 2025)', fontsize=14, fontweight='bold')
axes[0].set_ylim(0, 100)
axes[0].grid(axis='y', alpha=0.3)

# 작업 능력
models = ["Base", "SFT"]
math_es = [16, 52]
math_en = [20, 65]
x = range(len(models))
width = 0.35
axes[1].bar([i - width/2 for i in x], math_es, width, label='스페인어', color='#ff7f0e')
axes[1].bar([i + width/2 for i in x], math_en, width, label='영어', color='#2ca02c')
axes[1].set_ylabel('정확도 (%)', fontsize=12)
axes[1].set_title('수학 문제 해결 능력', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(models)
axes[1].set_ylim(0, 100)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 5.2 사용 사례별 권장사항

In [ ]:
recommendations = {
    "CPT 사용 권장": [
        "✅ 정적인 대규모 도메인 데이터 존재",
        "✅ 초저지연 필요",
        "✅ 컨텍스트 길이 축소 필요",
        "✅ 고도의 전문성 필요",
        "❌ 대부분의 경우 RAG가 더 적합",
    ],
    "SFT 사용 권장": [
        "✅ 특정 작업 능력 향상 필요",
        "✅ 언어 적응 필요",
        "✅ 추론 패턴 도입",
        "✅ 문체 제어 필요",
        "✅ 가장 범용적으로 유용",
    ],
    "DPO 사용 권장": [
        "✅ 스타일/어조 정렬",
        "✅ 안전성 향상",
        "✅ 선호도 학습",
        "✅ RLHF 대안으로 간단",
        "✅ 마지막 단계로 적용",
    ],
}

print("\n" + "="*80)
print("사용 사례별 권장사항")
print("="*80)

for category, items in recommendations.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  {item}")

### 5.3 전체 파이프라인 요약

In [ ]:
print("""
==================================================================================
NVIDIA NeMo 파인튜닝 파이프라인 요약
==================================================================================

1. 지속적 사전 학습 (CPT)
   목적: 도메인 지식 주입
   데이터: 대량의 원시 텍스트
   결과: GTC 2025 지식 습득 (55% → 59%)
   주의: 파국적 망각 위험, 대부분의 경우 RAG 권장

2. 지도 미세 조정 (SFT)
   목적: 작업 능력 향상
   데이터: 입력-출력 쌍
   결과: 스페인어 수학 52% (16% 대비 +36%p)
   특징: 교차 언어 전이 발생

3. 직접 선호도 최적화 (DPO)
   목적: 인간 선호도 정렬
   데이터: 선호/비선호 쌍
   결과: PT-PT 언어 변형 선호도 학습
   장점: RLHF보다 간단하고 효율적

==================================================================================
권장 워크플로우
==================================================================================

일반적인 경우:
  Base Model → SFT → DPO → Production

도메인 전문성 필요 시:
  Base Model → CPT → SFT → DPO → Production

빠른 프로토타이핑:
  Base Model → RAG (CPT 대신)

==================================================================================
""")

## 마무리

### 학습 내용 정리

이 노트북을 통해 다음을 학습했습니다:

1. **NeMo Run 프레임워크**
   - 레시피 기반 실험 구성
   - 분산 학습 관리
   - 체크포인트 및 로깅

2. **지속적 사전 학습 (CPT)**
   - 도메인 지식 주입
   - 데이터 토큰화
   - CPT vs. RAG 비교

3. **지도 미세 조정 (SFT)**
   - 작업 능력 향상
   - 언어 적응
   - 교차 언어 전이

4. **직접 선호도 최적화 (DPO)**
   - 선호도 정렬
   - NeMo-RL 사용
   - RLHF 대안

### 다음 단계

- 개별 노트북으로 각 기법 심화 학습
- 자신의 데이터로 실험
- 프로덕션 배포 고려
- 하이퍼파라미터 튜닝